## Introduction
In this tutorial, you'll learn about two common manipulations for geospatial data: **geocoding** and **table joins**.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker

# Function for displaying the map
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

## Geocoding
**Geocoding** is the process of converting the name of a place or an address to a location on a map. If you have ever looked up a geographic location based on a landmark description with Google Maps, Bing Maps, or Baidu Maps, for instance, then you have used a geocoder!

![Geocoding](https://i.imgur.com/1IrgZQq.png)

We'll use geopandas to do all of our geoencoding

In [3]:
from geopandas.tools import geocode

To use the geocoder, we need only provide:

- the name or address as a Python string, and
- the name of the provider; to avoid having to provide an API key, we'll use the OpenStreetMap Nominatim geocoder.

If the geocoding is successful, it returns a GeoDataFrame with two columns:

- the "geometry" column contains the (latitude, longitude) location, and
- the "address" column contains the full address.

In [7]:
result = geocode("The Great Pyramid of Giza", provider="nominatim")
result

,geometry,address
0,POINT (31.13424 29.97913),"هرم خوفو, Cause way, كوم الأخضر, الجيزة, محافظ..."


The entry in the "geometry" column is a Point object, and we can get the latitude and longitude from the y and x attributes, respectively.

In [8]:
point = result.geometry.iloc[0]
print("Latitude:", point.y)
print("Longitude:", point.x)

Latitude: 29.9791264
Longitude: 31.1342383751015


It's often the case that we'll need to geocode many different addresses. For instance, say we want to obtain the locations of 100 top universities in Europe.

In [11]:
universities = pd.read_csv("geospatial-learn-course-data/top_universities.csv")
universities.head(5)

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


Then we can use a lambda function to apply the geocoder to every row in the DataFrame. (We use a try/except statement to account for the case that the geocoding is unsuccessful.)

In [12]:
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None

universities[['Latitude', 'Longitude', 'geometry']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

# Drop universities that were not successfully geocoded
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(universities, geometry=universities.geometry)
universities.crs = {'init': 'epsg:4326'}
universities.head()

89.0% of addresses were geocoded!


,Name,Latitude,Longitude,geometry
0,University of Oxford,51.753454,-1.254010,POINT (-1.25401 51.75345)
1,University of Cambridge,52.176390,0.143089,POINT (0.14309 52.17639)
2,Imperial College London,51.498871,-0.175608,POINT (-0.17561 51.49887)
3,ETH Zurich,47.376453,8.547709,POINT (8.54771 47.37645)
4,UCL,51.521682,-0.135208,POINT (-0.13521 51.52168)


Next, we visualize all of the locations that were returned by the geocoder. Notice that a few of the locations are certainly inaccurate, as they're not in Europe!

In [17]:
# Create a map
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m)

# Display the map
embed_map(m, 'm.html')


## Table joins
Now, we'll switch topics and think about how to combine data from different sources.



**Attribute join**

You already know how to use `pd.DataFrame.join()` to combine information from multiple DataFrames with a shared index. We refer to this way of joining data (by simpling matching values in the index) as an **attribute join**.

When performing an attribute join with a GeoDataFrame, it's best to use the `gpd.GeoDataFrame.merge()`. To illustrate this, we'll work with a GeoDataFrame `europe_boundaries` containing the boundaries for every country in Europe. The first five rows of this GeoDataFrame are printed below.

In [18]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world.loc[world.continent == 'Europe'].reset_index(drop=True)

europe_stats = europe[["name", "pop_est", "gdp_md_est"]]
europe_boundaries = europe[["name", "geometry"]]

In [19]:
europe_stats.head()

,name,pop_est,gdp_md_est
0,Russia,142257519,3745000.0
1,Norway,5320045,364700.0
2,France,67106161,2699000.0
3,Sweden,9960487,498100.0
4,Belarus,9549747,165400.0


We do the attribute join in the code cell below. The on argument is set to the column name that is used to match rows in europe_boundaries to rows in `europe_stats`.

In [20]:
# Use an attribute join to merge data about countries in Europe
europe = europe_boundaries.merge(europe_stats, on="name")
europe.head()

,name,geometry,pop_est,gdp_md_est
0,Russia,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516...",142257519,3745000.0
1,Norway,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ...",5320045,364700.0
2,France,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ...",67106161,2699000.0
3,Sweden,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300...",9960487,498100.0
4,Belarus,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372...",9549747,165400.0
